# 08 - Rendering and Visualization

Generate novel view videos (GIF/MP4) for all trained models.

In [ ]:
import os
import numpy as np
import imageio
import time
import torch
from tqdm.auto import tqdm
from pathlib import Path

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Test Poses

In [ ]:
# Use test poses if available, otherwise train poses
json_path = DATA_DIR / 'transforms_test.json'
if not json_path.exists():
    json_path = DATA_DIR / 'transforms_train.json'

meta = jload(json_path)
frames = meta['frames']

print(f'Loaded {len(frames)} frames for rendering')

## Rendering Function

In [ ]:
@torch.no_grad()
def render_image(model, c2w, H, W, focal, chunk=4096):
    model.eval()
    
    c2w_tensor = torch.from_numpy(np.array(c2w, dtype=np.float32)).to(device)
    rays_o, rays_d = get_rays(H, W, focal, c2w_tensor)
    
    rays_o = rays_o.view(-1, 3)
    rays_d = rays_d.view(-1, 3)
    
    outs = []
    for i in range(0, rays_o.shape[0], chunk):
        rgb, _ = render_rays(model, rays_o[i:i+chunk], rays_d[i:i+chunk],
                            near=2.0, far=6.0, n_samples=128, perturb=False)
        outs.append(rgb)
    
    img = torch.cat(outs, 0).view(H, W, 3).cpu().numpy()
    return (img * 255).astype('uint8')

## Render All Models

In [ ]:
models = {
    'baseline': model_baseline,
    'soft': model_soft,
    'hard': model_hard,
    'hybrid': model_hybrid
}

for model_name, model in models.items():
    print(f'\nRendering {model_name}...')
    
    # Create output directory
    output_dir = Path(f'results/{model_name}/renders')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    gif_path = output_dir / f'{model_name}.gif'
    mp4_path = output_dir / f'{model_name}.mp4'
    
    # Create writers
    gif_writer = imageio.get_writer(gif_path, mode='I', duration=0.04, loop=0)
    mp4_writer = imageio.get_writer(mp4_path, fps=25)
    
    pbar = tqdm(total=len(frames), desc=f'Rendering {model_name}', unit='frame')
    
    for k, frame in enumerate(frames):
        t0 = time.time()
        
        img = render_image(model, frame['transform_matrix'], H, W, focal)
        
        gif_writer.append_data(img)
        mp4_writer.append_data(img)
        
        dt_ms = int((time.time() - t0) * 1000)
        pbar.set_postfix_str(f'{k+1}/{len(frames)} | {dt_ms} ms')
        pbar.update(1)
    
    pbar.close()
    gif_writer.close()
    mp4_writer.close()
    
    print(f'✅ Saved: {gif_path}')
    print(f'✅ Saved: {mp4_path}')

print('\n✅ All renderings complete!')

## Side-by-Side Comparison (Optional)

In [ ]:
# Create side-by-side comparison video
import matplotlib.pyplot as plt

print('Creating comparison video...')

comparison_path = Path('results/comparison.mp4')
comparison_writer = imageio.get_writer(comparison_path, fps=25)

for k, frame in enumerate(tqdm(frames, desc='Comparison')):
    fig, axes = plt.subplots(2, 2, figsize=(10, 10))
    fig.suptitle(f'Frame {k+1}/{len(frames)}', fontsize=16)
    
    for idx, (name, model) in enumerate(models.items()):
        img = render_image(model, frame['transform_matrix'], H, W, focal)
        ax = axes[idx // 2, idx % 2]
        ax.imshow(img)
        ax.set_title(name.capitalize())
        ax.axis('off')
    
    plt.tight_layout()
    
    # Convert figure to image
    fig.canvas.draw()
    img_array = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img_array = img_array.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    
    comparison_writer.append_data(img_array)
    plt.close(fig)

comparison_writer.close()
print(f'✅ Saved: {comparison_path}')

## Summary

In [ ]:
print('\n' + '='*60)
print('RENDERING COMPLETE!')
print('='*60)
print('\nGenerated files:')
print('  - results/baseline/renders/baseline.gif & .mp4')
print('  - results/soft/renders/soft.gif & .mp4')
print('  - results/hard/renders/hard.gif & .mp4')
print('  - results/hybrid/renders/hybrid.gif & .mp4')
print('  - results/comparison.mp4 (side-by-side)')
print('\n✅ All done!')